# TorsionDrive Datasets

An individual TorsionDrive is specific workflow in the QCArchive ecosystem that computes the energy profile of rotatable dihedral (torsion) for use in Force Field fitting. This workflow has been developed by the QCArchive Team in conjunction with:

 - [Lee-Ping Wang](https://chemistry.ucdavis.edu/people/lee-ping-wang)
 - [John Chodera](http://www.choderalab.org)
 - Chaya Stern
 - Yudong Qiu
 - [The Open Force Field Initiative](https://openforcefield.org)

The top-level TorsionDrive code can be found [here](https://github.com/lpwgroup/torsiondrive).

The TorsionDrive Dataset organizes many TorsionDrives together for data exploration, analysis, and methodology comparison. To begin, we can connect to the MolSSI QCArchive server and query all known TorsionDrive Datasets:

In [1]:
import qcportal as ptl
client = ptl.FractalClient()
client

FractalClient(server_name='The MolSSI QCArchive Server', address='https://api.qcarchive.molssi.org:443/', username='None')

In [2]:
client.list_collections("TorsionDriveDataset")

tagline
collection          name                                                                                  
TorsionDriveDataset Fragment Stability Benchmark                                                      None
                    OpenFF Fragmenter Phenyl Benchmark       Phenyl substituent torsional barrier heights.
                    OpenFF Full TorsionDrive Benchmark 1                                              None
                    OpenFF Group1 Torsions                                                            None
                    OpenFF Primary TorsionDrive Benchmark 1                                           None
                    OpenFF Substituted Phenyl Set 1                                                   None
                    Pfizer Discrepancy Torsion Dataset 1                                              None
                    SMIRNOFF Coverage Torsion Set 1                                                   None
                    TorsionDrive Paper                                                                None

One of the datasets can be obtained in the canonical manner:

In [3]:
ds = client.get_collection("TorsionDriveDataset", "OpenFF Fragmenter Phenyl Benchmark")

## Exploring the Dataset

Each row of the dataset is comprised of a `Entry` which contains a list of starting molecules for the TorsionDrive, the dihedral of interest (in this case zero-indexed), and the scan resolution of the dihedral. For the purposes of the underlying DataFrame each row is the name of this `Entry`.

In [4]:
ds.df.head()

""
c1c[cH:1][c:2](cc1)[C:3](=[O:4])O
c1[cH:1][c:2](cnc1)[C:3](=[O:4])O
[cH:1]1cncc[c:2]1[C:3](=[O:4])O
[cH:1]1cc(nc[c:2]1[C:3](=[O:4])O)[O-]
Cc1c[cH:1][c:2](cn1)[C:3](=[O:4])O


New computations are based off specifications which contain many additional parameters to tune the torsiondrive as well as the underlying computational method. Here, we can list all specifications that are attributed to this dataset.

In [5]:
ds.list_specifications()

,Description
Name,
UFF,UFF gradient evaluation with RDKit
B3LYP-D3,B3LYP-D3 evaluation with Psi4


As we can see, we have several specifications at different levels of theory. It is important to recall that these Collections are "live": new specifications can be added and individual TorsionDrives can be under computation. To see the current status of each specification the `status` function is provided:

In [6]:
ds.status(["uff", "b3lyp-d3"])

,UFF,B3LYP-D3
COMPLETE,165,226
INCOMPLETE,62,1


In [7]:
ds.status(["b3lyp-d3"], collapse=False, status="COMPLETE").head()

,B3LYP-D3
c1c[cH:1][c:2](cc1)[C:3](=[O:4])O,COMPLETE
c1[cH:1][c:2](cnc1)[C:3](=[O:4])O,COMPLETE
[cH:1]1cncc[c:2]1[C:3](=[O:4])O,COMPLETE
[cH:1]1cc(nc[c:2]1[C:3](=[O:4])O)[O-],COMPLETE
Cc1c[cH:1][c:2](cn1)[C:3](=[O:4])O,COMPLETE


## Visualizing the TorsionDrives

TorsionDrives can be visualized via the ``visualize`` command. Multiple torsiondrives can be plotted on the same graph for comparison.

In [8]:
ds.visualize(["[CH3:4][O:3][c:2]1[cH:1]cccc1", "[CH3:4][O:3][c:2]1[cH:1]ccnc1"], "B3LYP-D3", units="kJ / mol")

## Computational Requirements

We can check the number of optimizations and individual gradient evaluations by using the ``count`` method. 
By default, the ``count`` method shows the number of individual geometry optimizations:

In [9]:
ds.counts(["[CH3:4][O:3][c:2]1[cH:1]cccc1", "[CH3:4][O:3][c:2]1[cH:1]ccnc1"])

,UFF,B3LYP-D3
[CH3:4][O:3][c:2]1[cH:1]cccc1,49,49
[CH3:4][O:3][c:2]1[cH:1]ccnc1,49,53


In addition to individual optimization runs, we can also find a sum of how many gradient evaluations were performed in the course of the run.

In [10]:
ds.counts(["[CH3:4][O:3][c:2]1[cH:1]cccc1", "[CH3:4][O:3][c:2]1[cH:1]ccnc1"], count_gradients=True)

,UFF,B3LYP-D3
[CH3:4][O:3][c:2]1[cH:1]cccc1,606,601
[CH3:4][O:3][c:2]1[cH:1]ccnc1,538,680


## Deep data inspection

The TorsionDriveDataset also allows exploration of every single computation and molecule created during the individual TorsionDrive executions. Examples below this point will only be for those who wish to explore all of the individual computations in a TorsionDrive Dataset.

These TorsionDrive Datasets are different from canonical ``ReactionDataset`` and ``Dataset`` collections as each item in the underlying Pandas DataFrame is a TorsionDriveRecord object which contains links to all data used in the TorsionDrive. We can observe the these in the underlying DataFrame:

In [11]:
ds.df.head()

,UFF,B3LYP-D3
c1c[cH:1][c:2](cc1)[C:3](=[O:4])O,id=1761595 hash_index=973a82dee50895e16d0a1099...,id=1761822 hash_index=028379f6fcec47dd5a41b7c4...
c1[cH:1][c:2](cnc1)[C:3](=[O:4])O,id=1761596 hash_index=52cd24c390a25eee94e2ce1b...,id=1761823 hash_index=205e930c36ae28eb2dd5153d...
[cH:1]1cncc[c:2]1[C:3](=[O:4])O,id=1761597 hash_index=50db63b5d61dbaa0b326d588...,id=1761824 hash_index=381175f46c75a36d1bdec6c2...
[cH:1]1cc(nc[c:2]1[C:3](=[O:4])O)[O-],id=1761598 hash_index=aa028d05f9aac9984667e4e2...,id=1761825 hash_index=f11d261f1ba21ac280706a36...
Cc1c[cH:1][c:2](cn1)[C:3](=[O:4])O,id=1761599 hash_index=16aeb7c248405ebd9ea1d117...,id=1761826 hash_index=93cf4b08356a79aef7f70be6...


We can then begin to explore an individual TorsionDriveRecord by first pulling it from the DataFrame:

In [12]:
td = ds.df.loc["[CH3:4][O:3][c:2]1[cH:1]cccc1", "B3LYP-D3"]

We can then request a variety of attributes off this TorsionDriveRecord:

In [13]:
print("Torsion of interest                : {}".format(td.keywords.dihedrals))
print("Final optimization energy in hartree: {}".format(td.get_final_energies(180)))

Torsion of interest                : [(3, 5, 7, 6)]
Final optimization energy in hartree: -346.5319986074462


In [14]:
td.get_final_molecules(90)

_ColormakerRegistry()

NGLWidget()

The `Molecule` object has a `measure` attribute so that we check the dihedral angle is in fact 180 degrees.

In [15]:
"3-5-7-6 dihedral in degrees: {}".format(td.get_final_molecules(180).measure([3, 5, 7, 6]))

'3-5-7-6 dihedral in degrees: 179.99999995886662'

You can also get a dictionary of all final molecules:

In [16]:
td.get_final_molecules()

{(-90,): Molecule(schema_name=qcschema_molecule, schema_version=2, validated=True, symbols=['C' 'C' 'C' 'C' 'C' 'C' 'C' 'O' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'], geometry=[[ 3.40718507 -2.21840696 -0.22958297]
  [ 4.97493788 -3.48334777  1.48334476]
  [ 1.7716505  -0.3298455   0.63640014]
  [ 4.90787611 -2.87176474  4.05190433]
  [ 1.69441113  0.29082009  3.20260133]
  [ 3.26680019 -0.98070424  4.91013927]
  [ 1.5337185  -1.70492748  8.94696498]
  [ 3.24498132 -0.32624151  7.42021119]
  [ 3.46717648 -2.69676568 -2.23750076]
  [ 6.26600112 -4.95093418  0.81459893]
  [ 0.55229994  0.67321911 -0.69586245]
  [ 6.13043938 -3.82147864  5.41754483]
  [ 0.44593371  1.77186336  3.91617642]
  [ 1.72079846 -0.98831032 10.88831694]
  [ 1.96264592 -3.7518771   8.93520479]
  [-0.44384326 -1.44836098  8.3163972 ]], name=C7H8O, identifiers={'molecule_hash': '34e0a1476c201686f17fe576609083dc7d8bb187', 'molecular_formula': 'C7H8O', 'smiles': None, 'inchi': None, 'inchikey': None, 'canonical_explicit_hydroge

## Exploring connection optimizations

If desired, we can pull each geometry optimization belonging to the torsiondrive with the `get_history` function. 
In this case, we will pull the lowest energy optimization for the 180 degree dihedral:

In [17]:
opt = td.get_history(180, minimum=True)
opt

OptimizationRecord(id=1701507, hash_index=effb8d6307ac3cc1a263e674e93342498f10219f, procedure=optimization, program=geometric, version=1, protocols={}, extras={}, stdout=228720, stderr=None, error=None, task_id=5c988a95b6a2de5fb9b9e33f, manager_name=None, status=COMPLETE, modified_on=2019-03-25 08:02:57.063000, created_on=2019-03-22 20:21:05.145000, provenance={'creator': 'geomeTRIC', 'version': '0.9.5', 'routine': 'geometric.run_json.geometric_run_json', 'wall_time': 111.76719880104065, 'qcengine_version': 'v0.6.4', 'cpu': 'Intel(R) Xeon(R) CPU E5-2680 v3 @ 2.50GHz', 'username': 'lnaden', 'hostname': 'dt039'}, schema_version=1, initial_molecule=458443, qc_spec={'driver': <DriverEnum.gradient: 'gradient'>, 'method': 'b3lyp-d3', 'basis': 'def2-svp', 'keywords': '5c954fa6b6a2de5f188ea234', 'program': 'psi4'}, keywords={'coordsys': 'tric', 'enforce': 0.1, 'constraints': {'set': [{'type': 'dihedral', 'indices': [3, 5, 7, 6], 'value': 180}]}, 'program': 'psi4'}, energies=[-346.5314762261408

In [18]:
opt.energies

[-346.53147622614085,
 -346.53177915374135,
 -346.5318873591642,
 -346.53194287295247,
 -346.53197743434515,
 -346.5319973422999,
 -346.53199831291954,
 -346.53199826976714,
 -346.5319986074462]

## Exploring individual gradient evaluations

We can go even deeper in the calculations to look at each gradient calculation of the `Optimization` calculation if we so choose and see even more details about how the `TorsionDrive` object was constructed.

In [19]:
result = opt.get_trajectory()[-1]

In [20]:
print("Program:                   {}".format(result.program))
print("Number of Basis Functions: {}".format(result.properties.calcinfo_nbasis))
print("Total execution time:      {:.2f}s".format(result.provenance.wall_time))

Program:                   psi4
Number of Basis Functions: 152
Total execution time:      12.15s


This example also contains the Wiberg-Lowdin indices. 
As this is specific to Psi4, this data resides inside the `extras` tag rather than general properties. 
This data is not yet well curated and currently exists as a 1D list. 
We will first pull this data and transform it to a 2D array:

In [21]:
import numpy as np
wiberg = np.array(result.extras["qcvars"]["WIBERG_LOWDIN_INDICES"]).reshape(-1, 16)

As this particular example is exploring the `3-5-7-6` dihedral, we would find the most use in the `5-7` bond. This can be acquired as follows: 

In [22]:
wiberg[5, 7]

1.2700940280530457

We can continue to explore these results and even obtain the standard Psi4 logging information!

In [23]:
print(result.get_stdout()[:1000])


  Memory set to  60.800 GiB by Python driver.
gradient() will perform analytic gradient computation.

*** tstart() called on dt039
*** at Mon Mar 25 04:02:23 2019

   => Loading Basis Set <=

    Name: DEF2-SVP
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-7  entry C          line    90 file /home/lnaden/miniconda3/envs/qca/share/psi4/basis/def2-svp.gbs 
    atoms 8    entry O          line   130 file /home/lnaden/miniconda3/envs/qca/share/psi4/basis/def2-svp.gbs 
    atoms 9-16 entry H          line    15 file /home/lnaden/miniconda3/envs/qca/share/psi4/basis/def2-svp.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        6 Threads,  62259 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular 